In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.nhl_db
collection = db.articles

In [8]:
# URL of page to be scraped
url = 'https://www.nhl.com/news'

In [9]:
# Retrieve page with the requests module
response = requests.get(url)

In [10]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [7]:
soup

<!DOCTYPE html>
<html lang="en_US">
<head>
<title>Official Site of the National Hockey League | NHL.com </title>
<!-- meta meta tag -->
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="en" http-equiv="content-language"/>
<meta content="nhl, nhl.com, www.nhl.com, playoffs, scores, video, photos, standings, news, features, players, shop, auctions, tickets, mobile, game center live, stanley cup, winter classic, draft, free agency" name="keywords"/>
<meta content="US" name="countryCode"/>
<meta content="Official Site of the National Hockey League" property="og:title"/>
<meta content="Official Site of the National Hockey League NHL.com" itemprop="name"/>
<meta content="NHL.com" property="og:site_name"/>
<meta content="website" property="og:type"/>
<meta content=

In [24]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='article-item__top')
# Loop through results to retrieve article title, header, and timestamp of article
for result in results:
    print(result.find('h1', class_='article-item__headline').text)
    dati = result.find('span', class_='article-item__date')['data-date']
    print(dati)
    print(dati.split('T')[0])
    print(dati.split('T')[1].split('-')[0])

Hynes fired as coach of Devils, replaced by Nasreddine
2019-12-03T19:09:36-0500
2019-12-03
19:09:36
Devils lose to Golden Knights in first game after Hynes fired
2019-12-03T21:50:36-0500
2019-12-03
21:50:36
Bruins shut out Hurricanes, get eighth straight victory
2019-12-03T21:46:29-0500
2019-12-03
21:46:29
Flyers pull away from Maple Leafs at end, get fifth straight win
2019-12-03T21:37:27-0500
2019-12-03
21:37:27
Fantasy defenseman top 50 rankings for 2019-20
2019-12-03T16:20:40-0500
2019-12-03
16:20:40
Wild rally past Panthers in fourth straight win, extend point streak
2019-12-03T21:46:39-0500
2019-12-03
21:46:39
Soderberg has goal, two assists in Coyotes win against Blue Jackets
2019-12-03T21:46:05-0500
2019-12-03
21:46:05
NHL Commissioner Bettman, Deputy Commissioner Daly statement on Akim Aliu
2019-12-03T16:53:44-0500
2019-12-03
16:53:44
Keith to miss at least next two games for Blackhawks
2019-12-03T15:52:37-0500
2019-12-03
15:52:37
NHL DFS picks, projections, strategy
2019-12-0

In [28]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='article-item__top')
# Loop through results to retrieve article title, header, and timestamp of article
for result in results:
    try:
        title = result.find('h1', class_='article-item__headline').text

        lede = result.find('h2', class_='article-item__subheader').text

        # The time and date of article publication

        date_time = result.find('span', class_='article-item__date')['data-date']

        # Slice the datetime string for the date

        date = date_time.split('T')[0]

        # Slice the datetime string for the time

        time = date_time.split('T')[1].split('-')[0]

        # Determine whether article was published in AM or PM
        if (int(time[:2]) >= 13):
            meridiem = 'pm'
        else:
            meridiem = 'am'

        # Concatenate time string
        time = time + meridiem
        print('-----------------')
        print(title)
        print(lede)
        print(date)
        print(time)

        # Dictionary to be inserted into MongoDB

        post = {
            'title': title,
            'lede': lede,
            'date': date,
            'time': time
        }

        # Insert dictionary into MongoDB as a document
        
        collection.insert_one(post)
        
    except Exception as e:
        print(e)

-----------------
Hynes fired as coach of Devils, replaced by Nasreddine
New Jersey next-to-last in Eastern Conference
2019-12-03
19:09:36pm
-----------------
Devils lose to Golden Knights in first game after Hynes fired
Nasreddine defeated in New Jersey debut; Marchessault has third-period hat trick for Vegas
2019-12-03
21:50:36pm
-----------------
Bruins shut out Hurricanes, get eighth straight victory
Halak makes 24 saves, Boston breaks tie late in third to extend point streak to 12
2019-12-03
21:46:29pm
-----------------
Flyers pull away from Maple Leafs at end, get fifth straight win
Konecny has goal, assist in third period after Toronto ties game
2019-12-03
21:37:27pm
-----------------
Fantasy defenseman top 50 rankings for 2019-20
Trouba, Fox, DeAngelo providing value from Rangers; Karlsson climbing list for Sharks
2019-12-03
16:20:40pm
-----------------
Wild rally past Panthers in fourth straight win, extend point streak
Kahkonen makes 44 saves for Minnesota, which is 7-0-3 in 

In [ ]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)